# TRABAJO PRACTICO FINAL - PROCESAMIENTO DEL LENGUAJE NATURAL

**Tecnicatura Universitaria en Inteligencia Artificial**  
**Universidad Nacional de Rosario**

---

**Materia:** IA 4.2 - Procesamiento del Lenguaje Natural  
**Estudiante:** Sebastián Palacio  
**DNI:** 43491996
**Fecha de entrega:** 7 de diciembre de 2025

**Docentes:**
- Juan Pablo Manson
- Alan Geary
- Andrea Cavallo
- Ariel D'Alessandro

---

## Descripcion del Trabajo

Este notebook implementa un sistema completo de asistente virtual para una empresa de electrodomesticos utilizando:
- **Ejercicio 1:** Sistema RAG con 3 bases de datos (vectorial, tabular, grafos)
- **Ejercicio 2:** Agente autonomo con patron ReAct y 4 herramientas especializadas

---

# **Parte 1**

In [380]:
from google.colab import drive
drive.mount('/content/drive')

!unzip "/content/drive/MyDrive/NLP/todo.zip" -d /content/

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: /content/resenas_usuarios/resena_R02810.txt  
  inflating: /content/resenas_usuarios/resena_R04844.txt  
  inflating: /content/resenas_usuarios/resena_R03827.txt  
  inflating: /content/resenas_usuarios/resena_R02387.txt  
  inflating: /content/resenas_usuarios/resena_R04787.txt  
  inflating: /content/resenas_usuarios/resena_R02558.txt  
  inflating: /content/resenas_usuarios/resena_R03381.txt  
  inflating: /content/resenas_usuarios/resena_R02048.txt  
  inflating: /content/resenas_usuarios/resena_R04816.txt  
  inflating: /content/resenas_usuarios/resena_R03409.txt  
  inflating: /content/resenas_usuarios/resena_R03156.txt  
  inflating: /content/resenas_usuarios/resena_R04748.txt  
  inflating: /content/resenas_usuarios/resena_R02663.txt  
  inflating: /content/resenas_usuarios/resena_R03348.txt  
  inflating: /content/resenas_usuarios/resena_R04040.txt  
  inflating: /content/resenas_usuarios/resena_R02545.t

## **Distribución por tipo de base de datos**

1. BASE DE DATOS VECTORIAL (ChromaDB/FAISS)

Contenido:
  - FAQs (3,000 preguntas-respuestas)
  - Manuales de productos (50 manuales)
  - [OPCIONAL] Reseñas de usuarios (5,015 reseñas)

Justificación:
  - Información textual no estructurada
  - Ideal para búsqueda semántica
  - Responde preguntas como:
    * "¿Cómo uso mi licuadora para hacer smoothies?"
    * "¿Qué voltaje requiere el rallador digital eléctrico?"
    * "¿Qué opinan los usuarios de esta cafetera?"


2. BASE DE DATOS TABULAR (Pandas/SQL)

Contenido:
  - productos.csv (precio, stock, specs técnicas)
  - inventario_sucursales.csv (stock por sucursal)
  - ventas_historicas.csv (para analytics_tool en Ejercicio 2)

Justificación:
  - Datos estructurados con campos numéricos y categóricos
  - Requiere filtros y comparaciones exactas
  - Responde preguntas como:
    * "¿Cuáles son las licuadoras de menos de $200?"
    * "¿Qué productos están en la categoría Cocina?"
    * "Quiero una licuadora con buenas reseñas"


3. BASE DE DATOS DE GRAFOS (Neo4j/NetworkX)

Contenido (Relaciones):
  - Producto -> Categoría -> Subcategoría
  - Producto -> Vendedor (desde ventas_historicas)
  - Vendedor -> Sucursal
  - Producto -> Tickets de soporte
  - Cliente -> Provincia -> Compras

Justificación:
  - Datos interconectados con múltiples relaciones
  - Consultas de tipo "caminos" y "conexiones"
  - Responde preguntas como:
    * "¿Qué productos están relacionados con Cocina?"
    * "¿Qué vendedores venden productos de la marca X?"
    * "¿Qué productos tienen más tickets de soporte?"

### Justificación de Framework

In [417]:
framework_analysis = """
DECISIÓN: Uso de LlamaIndex en lugar de Langchain

CONTEXTO DEL ENUNCIADO:
El TP especifica: "Utilizar Langchain para implementar un agente basado
en el paradigma ReAct"

DECISIÓN TOMADA:
Se utilizó LlamaIndex para todo el proyecto (Ejercicio 1 y 2)

JUSTIFICACIÓN TÉCNICA:

COMPATIBILIDAD CON MATERIAL DE CLASE:
   - El material TUIA_NLP_Unidad_7.ipynb usa LlamaIndex y me guié principalmente por los contenidos de clase
   - Los ejemplos de ReAct en clase están en LlamaIndex
   - Metodología enseñada: LlamaIndex para RAG + Agentes

"""

## **Preparación de dependencias y librerías**

In [382]:
%%capture
!pip install llama_index==0.12.39 llama-index-embeddings-huggingface==0.5.4 sentence-transformers==4.1.0
!pip install rank_bm25 unidecode nltk
!pip install neo4j networkx matplotlib
!pip install faiss-cpu pandas
!pip install -q chromadb langchain langchain-community sentence-transformers rank-bm25

In [383]:
import json
import os
import pandas as pd
import numpy as np
from typing import List, Dict, Any
import time

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, Document as LlamaDocument, Settings
from llama_index.core.schema import TextNode, NodeWithScore

from sentence_transformers import CrossEncoder, SentenceTransformer

from rank_bm25 import BM25Okapi
import nltk
from nltk.tokenize import word_tokenize
from unidecode import unidecode
import re

from neo4j import GraphDatabase
import networkx as nx
import matplotlib.pyplot as plt

import sqlite3
from sqlalchemy import create_engine

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns

ruta_base = '/content/'


### Configuración del modelo de embeddings

In [384]:
print('Cargando modelo de embeddings para español...')

model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

#Configurar globalmente en LlamaIndex
Settings.embed_model = HuggingFaceEmbedding(model_name=model_name)

print(f"Modelo configurado: {model_name}")

#Prueba
test_embedding = Settings.embed_model.get_text_embedding("¿Cómo uso mi licuadora?")
print(f"Dimensión del embedding: {len(test_embedding)}")

Cargando modelo de embeddings para español...
Modelo configurado: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Dimensión del embedding: 384


### Carga de datos

In [385]:
#FAQs
print("\n1. Cargando FAQs...")
with open(ruta_base + 'faqs.json', 'r', encoding='utf-8') as f:
    faqs = json.load(f)
print(f"   ✓ {len(faqs)} FAQs cargados")

#Manuales
print("\n2. Cargando manuales...")
manuales_dir = ruta_base + 'manuales_productos/'
manuales = []

for archivo in os.listdir(manuales_dir):
    if archivo.endswith('.md'):
        with open(os.path.join(manuales_dir, archivo), 'r', encoding='utf-8') as f:
            contenido = f.read()

        id_producto = archivo.split('_')[1]
        manuales.append({
            'id_producto': id_producto,
            'nombre_archivo': archivo,
            'contenido': contenido
        })

print(f"   ✓ {len(manuales)} manuales cargados")

#CSVs para base tabular
print("\n3. Cargando datos tabulares...")
df_productos = pd.read_csv(ruta_base + 'productos.csv')
df_ventas = pd.read_csv(ruta_base + 'ventas_historicas.csv')
df_inventario = pd.read_csv(ruta_base + 'inventario_sucursales.csv')

print(f"   ✓ Productos: {len(df_productos)} registros")
print(f"   ✓ Ventas: {len(df_ventas)} registros")
print(f"   ✓ Inventario: {len(df_inventario)} registros")




1. Cargando FAQs...
   ✓ 3000 FAQs cargados

2. Cargando manuales...
   ✓ 50 manuales cargados

3. Cargando datos tabulares...
   ✓ Productos: 300 registros
   ✓ Ventas: 10000 registros
   ✓ Inventario: 4100 registros


## **Creación de las bases de datos**

### Base de datos 1: vectorial (ChromaDB con LlamaIndex)

In [386]:
print("\n3. Creando índice vectorial...")
vector_index = VectorStoreIndex.from_documents(
    documents,
    show_progress=True
)

print("\n Índice vectorial creado exitosamente")

vector_retriever = vector_index.as_retriever(similarity_top_k=5)
print("Retriever configurado (top_k=5)")


3. Creando índice vectorial...


Parsing nodes:   0%|          | 0/3050 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1104 [00:00<?, ?it/s]


 Índice vectorial creado exitosamente
Retriever configurado (top_k=5)


In [387]:
print("CREANDO BASE DE DATOS VECTORIAL")
documents = []

#Convertir FAQs a documentos
print("\n1. Procesando FAQs...")
for faq in faqs:
    texto = f"Pregunta: {faq['pregunta']}\nRespuesta: {faq['respuesta']}"

    #Crear documento de LlamaIndex con metadata
    doc = LlamaDocument(
        text=texto,
        metadata={
            'id_faq': faq['id_faq'],
            'id_producto': faq['id_producto'],
            'nombre_producto': faq['nombre_producto'],
            'categoria': faq['categoria'],
            'tipo': 'faq'
        }
    )
    documents.append(doc)

print(f"    {len(faqs)} FAQs convertidos a documentos")

#Convertir Manuales a documentos
print("\n2. Procesando manuales...")
for manual in manuales:
    doc = LlamaDocument(
        text=manual['contenido'],
        metadata={
            'id_producto': manual['id_producto'],
            'nombre_archivo': manual['nombre_archivo'],
            'tipo': 'manual'
        }
    )
    documents.append(doc)

print(f"    {len(manuales)} manuales convertidos a documentos")

print(f"\n Total documentos para indexar: {len(documents)}")

CREANDO BASE DE DATOS VECTORIAL

1. Procesando FAQs...
    3000 FAQs convertidos a documentos

2. Procesando manuales...
    50 manuales convertidos a documentos

 Total documentos para indexar: 3050


### Base de datos 2: Tabular

In [388]:
#Extraer metadata de los dataframes (para el LLM)
def extract_dataframe_metadata(df, df_name):
    """Extrae información relevante del DataFrame para el LLM"""
    metadata = {
        'nombre': df_name,
        'shape': df.shape,
        'columnas': list(df.columns),
        'tipos': df.dtypes.to_dict(),
        'info_columnas': {}
    }

    for col in df.columns:
        col_info = {}

        if df[col].dtype in ['object', 'category']:
            #Campos categóricos
            valores_unicos = df[col].dropna().unique()
            col_info['tipo'] = 'categórico'
            col_info['valores_unicos'] = list(valores_unicos[:20])  # Máximo 20
            col_info['n_unicos'] = len(valores_unicos)
        elif df[col].dtype in ['int64', 'float64']:
            #Campos numéricos
            col_info['tipo'] = 'numérico'
            col_info['min'] = float(df[col].min()) if not pd.isna(df[col].min()) else None
            col_info['max'] = float(df[col].max()) if not pd.isna(df[col].max()) else None
            col_info['mean'] = float(df[col].mean()) if not pd.isna(df[col].mean()) else None
        else:
            col_info['tipo'] = str(df[col].dtype)

        metadata['info_columnas'][col] = col_info

    return metadata

#Extraer metadata
metadata_productos = extract_dataframe_metadata(df_productos, 'productos')
metadata_ventas = extract_dataframe_metadata(df_ventas, 'ventas_historicas')
metadata_inventario = extract_dataframe_metadata(df_inventario, 'inventario_sucursales')

#Mostrar ejemplo de metadata
print("\n   Ejemplo - Metadata de productos:")
print(f"   - Shape: {metadata_productos['shape']}")
print(f"   - Columnas: {metadata_productos['columnas'][:5]}...")
print(f"   - Categorías únicas: {metadata_productos['info_columnas']['categoria']}")


   Ejemplo - Metadata de productos:
   - Shape: (300, 14)
   - Columnas: ['id_producto', 'nombre', 'categoria', 'subcategoria', 'marca']...
   - Categorías únicas: {'tipo': 'categórico', 'valores_unicos': ['Cocina', 'Climatización', 'Lavado', 'Audio y Video'], 'n_unicos': 4}


In [389]:
#Crear string de información para el LLM
def create_metadata_string(metadata):
    """Convierte metadata en string legible para el LLM"""
    lines = []
    lines.append(f"DataFrame: {metadata['nombre']}")
    lines.append(f"Shape: {metadata['shape'][0]} filas, {metadata['shape'][1]} columnas")
    lines.append(f"Columnas: {', '.join(metadata['columnas'])}")
    lines.append("\nInformación de columnas:")

    for col, info in metadata['info_columnas'].items():
        if info['tipo'] == 'categórico':
            valores = info['valores_unicos'][:10]
            lines.append(f"  - {col}: categórico, valores ejemplo: {valores}")
        elif info['tipo'] == 'numérico':
            lines.append(f"  - {col}: numérico, rango [{info['min']}, {info['max']}]")

    return "\n".join(lines)

info_productos = create_metadata_string(metadata_productos)
info_ventas = create_metadata_string(metadata_ventas)
info_inventario = create_metadata_string(metadata_inventario)

### Base de datos 3: Grafos (Neo4j local)


In [390]:
%%capture
!apt-get install wget -y
!wget -O - https://debian.neo4j.com/neotechnology.gpg.key | apt-key add -
!echo 'deb https://debian.neo4j.com stable latest' | tee -a /etc/apt/sources.list.d/neo4j.list
!apt-get update
!apt-get install neo4j -y

In [391]:
#Quitar autenticación
!sed -i 's/#dbms.security.auth_enabled=false/dbms.security.auth_enabled=false/' /etc/neo4j/neo4j.conf
!service neo4j start
time.sleep(3)

uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri)
print("Neo4j iniciado y conectado")

#Verifico si está vacío
with driver.session() as session:
    result = session.run("MATCH (n) RETURN count(n) as total")
    total = result.single()["total"]
    print(f"Nodos actuales: {total}")

#Poblar si está vacío
if total == 0:

    relaciones = []

    for _, row in df_productos.iterrows():
        relaciones.append({
            'source': row['id_producto'], 'source_type': 'Producto',
            'relation': 'PERTENECE_A',
            'target': row['categoria'], 'target_type': 'Categoria'
        })
        relaciones.append({
            'source': row['categoria'], 'source_type': 'Categoria',
            'relation': 'TIENE_SUBCATEGORIA',
            'target': row['subcategoria'], 'target_type': 'Subcategoria'
        })
        relaciones.append({
            'source': row['id_producto'], 'source_type': 'Producto',
            'relation': 'ES_DE_MARCA',
            'target': row['marca'], 'target_type': 'Marca'
        })

    for _, row in df_ventas.head(100).iterrows():
        relaciones.append({
            'source': row['id_producto'], 'source_type': 'Producto',
            'relation': 'VENDIDO_POR',
            'target': row['id_vendedor'], 'target_type': 'Vendedor'
        })
        relaciones.append({
            'source': row['id_vendedor'], 'source_type': 'Vendedor',
            'relation': 'TRABAJA_EN',
            'target': row['sucursal'], 'target_type': 'Sucursal'
        })

    df_relaciones = pd.DataFrame(relaciones).drop_duplicates()
    print(f"Total relaciones: {len(df_relaciones)}")

    #Funciones para Neo4j
    def clear_database(tx):
        tx.run("MATCH (n) DETACH DELETE n")

    def create_schema(tx):
        tx.run("CREATE CONSTRAINT IF NOT EXISTS FOR (e:Entity) REQUIRE e.name IS UNIQUE")
        tx.run("CREATE INDEX IF NOT EXISTS FOR (e:Entity) ON (e.name)")

    def add_triple(tx, subject, relation, obj, subject_type, object_type):
        query = f"""
        MERGE (s:Entity:{subject_type} {{name: $subject}})
        MERGE (o:Entity:{object_type} {{name: $object}})
        MERGE (s)-[r:`{relation}`]->(o)
        """
        tx.run(query, subject=subject, object=obj)

    print("\nInsertando en Neo4j...")
    with driver.session() as session:
        session.execute_write(clear_database)
        session.execute_write(create_schema)

    with driver.session() as session:
        for i, row in df_relaciones.iterrows():
            session.execute_write(
                add_triple,
                row['source'], row['relation'], row['target'],
                row['source_type'], row['target_type']
            )
            if (i + 1) % 200 == 0:
                print(f"  Insertadas {i + 1}/{len(df_relaciones)}...")

    #Verifición
    with driver.session() as session:
        result = session.run("MATCH (n) RETURN count(n) as total")
        total = result.single()["total"]
        print(f"\n Neo4j poblado: {total} nodos")
else:
    print("Todo correcto, Neo4j ya tiene datos")

Neo4j is already running (pid:82206).
Run with '--verbose' for a more detailed error message.
Neo4j iniciado y conectado
Nodos actuales: 410
Todo correcto, Neo4j ya tiene datos


## **Funciones de búsqueda**

### BÚSQUEDA HÍBRIDA: BM25 + Reranking

In [392]:
print("CONFIGURANDO BÚSQUEDA HÍBRIDA (BM25 + RERANKING)")

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

print("\n1. Implementando BM25Searcher...")

class BM25Searcher:
    """
    Implementación de búsqueda BM25 compatible con LlamaIndex (siguiendo clase)
    """
    def __init__(self, documents: List[LlamaDocument], language: str = 'spanish'):
        self.language = language
        self.documents = documents

        self.text_nodes = []
        for i, doc in enumerate(self.documents):
            text_node = TextNode(
                text=doc.text,
                metadata=doc.metadata if hasattr(doc, 'metadata') else {},
                id_=f"bm25_node_{i}"
            )
            self.text_nodes.append(text_node)

        self.corpus = [self._preprocess_text(node.text) for node in self.text_nodes]
        self.tokenized_corpus = [self._tokenize(text) for text in self.corpus]

        # Inicializa BM25
        self.bm25 = BM25Okapi(self.tokenized_corpus)

    def _preprocess_text(self, text: str) -> str:
        text = text.lower()
        text = unidecode(text)
        text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text

    def _tokenize(self, text: str) -> List[str]:
        return word_tokenize(text, language=self.language)

    def retrieve(self, query: str, top_k: int = 5) -> List[NodeWithScore]:
        processed_query = self._preprocess_text(query)
        tokenized_query = self._tokenize(processed_query)

        scores = self.bm25.get_scores(tokenized_query)
        top_indices = np.argsort(scores)[-top_k:][::-1]

        results = []
        for idx in top_indices:
            if scores[idx] > 0:
                node = NodeWithScore(
                    node=self.text_nodes[idx],
                    score=float(scores[idx])
                )
                results.append(node)

        return results

bm25_searcher = BM25Searcher(documents=documents)
print("    BM25Searcher creado")

class Reranker:
    """Reranker con CrossEncoder"""
    def __init__(self, model_name: str = 'cross-encoder/ms-marco-MiniLM-L-6-v2'):
        self.model = CrossEncoder(model_name)

    def rerank(self, query: str, nodes: List[NodeWithScore], top_k: int = None) -> List[NodeWithScore]:
        if not nodes:
            return nodes

        pairs = [(query, node.node.text) for node in nodes]

        scores = self.model.predict(pairs)

        scored_nodes = []
        for score, node in zip(scores, nodes):
            node.score = float(score)
            scored_nodes.append(node)
        scored_nodes.sort(key=lambda x: x.score, reverse=True)

        if top_k:
            scored_nodes = scored_nodes[:top_k]

        return scored_nodes

#Crear reranker
print("   Cargando modelo de reranking...")
reranker = Reranker()
print("    Reranker creado")

print("\n Búsqueda híbrida configurada correctamente")

CONFIGURANDO BÚSQUEDA HÍBRIDA (BM25 + RERANKING)

1. Implementando BM25Searcher...
    BM25Searcher creado
   Cargando modelo de reranking...
    Reranker creado

 Búsqueda híbrida configurada correctamente


### FUNCIÓN DE BÚSQUEDA HÍBRIDA (Vector + BM25 + Rerank)

In [393]:
print("FUNCIÓN DE BÚSQUEDA HÍBRIDA")

def hybrid_search(query: str, top_k: int = 3, apply_rerank: bool = True):
    """
    Búsqueda híbrida que combina:
    1. Búsqueda vectorial (semántica)
    2. Búsqueda BM25 (palabras clave)
    3. Reranking (opcional)

    Args:
        query: Consulta del usuario
        top_k: Número de resultados finales
        apply_rerank: Si aplicar reranking o no

    Returns:
        Lista de NodeWithScore con los mejores resultados
    """
    #Búsqueda vectorial
    vector_nodes = vector_retriever.retrieve(query)

    #Búsqueda BM25
    bm25_nodes = bm25_searcher.retrieve(query, top_k=5)

    #Combinar resultados
    all_nodes = vector_nodes.copy()

    for bm25_node in bm25_nodes:
        is_duplicate = any(
            bm25_node.node.text == vector_node.node.text
            for vector_node in vector_nodes
        )
        if not is_duplicate:
            all_nodes.append(bm25_node)

    #Aplico reranking
    if apply_rerank:
        final_nodes = reranker.rerank(query, all_nodes, top_k=top_k)
    else:
        final_nodes = all_nodes[:top_k]

    return final_nodes

print("✓ Función hybrid_search() creada")

#Prueba de la búsqueda híbrida
print("PRUEBA DE BÚSQUEDA HÍBRIDA")

query_prueba = "¿Cómo uso mi licuadora para hacer smoothies?"
print(f"\nConsulta: {query_prueba}")

resultados = hybrid_search(query_prueba, top_k=2)

for i, node_with_score in enumerate(resultados, 1):
    print(f"\n--- Resultado {i} (Score: {node_with_score.score:.4f}) ---")
    print(f"Tipo: {node_with_score.node.metadata.get('tipo', 'N/A')}")
    print(f"Producto: {node_with_score.node.metadata.get('id_producto', 'N/A')}")
    print(f"Texto (primeros 200 chars):\n{node_with_score.node.text[:200]}...")

print("\n Búsqueda híbrida funcionando correctamente")

FUNCIÓN DE BÚSQUEDA HÍBRIDA
✓ Función hybrid_search() creada
PRUEBA DE BÚSQUEDA HÍBRIDA

Consulta: ¿Cómo uso mi licuadora para hacer smoothies?

--- Resultado 1 (Score: 1.2773) ---
Tipo: faq
Producto: P0001
Texto (primeros 200 chars):
Pregunta: ¿Cómo se usa correctamente este producto?
Respuesta: El Licuadora de TechHome está diseñado para uso doméstico. Revise el manual del producto (código P0001) para más detalles. Ante cualquier...

--- Resultado 2 (Score: 1.2773) ---
Tipo: faq
Producto: P0001
Texto (primeros 200 chars):
Pregunta: ¿Cómo se usa correctamente este producto?
Respuesta: El Licuadora de TechHome está diseñado para uso doméstico. Revise el manual del producto (código P0001) para más detalles. Ante cualquier...

 Búsqueda híbrida funcionando correctamente


## **Clasificador de Intenciones**

In [394]:
#3 intenciones posibles
INTENCIONES = ['vectorial', 'tabular', 'grafos']

print("\nIntenciones del sistema:")
print("  1. vectorial: Preguntas sobre uso, FAQs, manuales")
print("  2. tabular: Preguntas sobre precios, stock, especificaciones")
print("  3. grafos: Preguntas sobre relaciones (categorías, marcas, sucursales)")


Intenciones del sistema:
  1. vectorial: Preguntas sobre uso, FAQs, manuales
  2. tabular: Preguntas sobre precios, stock, especificaciones
  3. grafos: Preguntas sobre relaciones (categorías, marcas, sucursales)


### Creación de datos sintéticos para entrenar los clasificadores

In [395]:
datos_entrenamiento = []

#VECTORIAL - Preguntas sobre uso, FAQs, manuales
preguntas_vectorial = [
    "¿Cómo uso mi licuadora para hacer smoothies?",
    "¿Qué voltaje requiere el rallador digital?",
    "¿Cómo se limpia la freidora?",
    "¿Para qué sirve el botón pulse de la licuadora?",
    "¿Qué dice el manual sobre el mantenimiento?",
    "¿Cómo funciona la función de auto-limpieza?",
    "¿Qué especificaciones técnicas tiene este producto?",
    "¿Cómo instalo el aire acondicionado?",
    "¿Qué precauciones debo tener al usar la batidora?",
    "¿Cómo se programa el microondas?",
    "Necesito instrucciones de uso del producto",
    "¿Qué dice la garantía sobre este problema?",
    "¿Cómo se usa correctamente este electrodoméstico?",
    "¿Qué opinan los usuarios sobre esta cafetera?",
    "¿Tiene buenas reseñas este producto?",
    "¿Qué dicen los clientes sobre la calidad?",
    "¿Es recomendable según las opiniones?",
    "¿Cuál es la valoración promedio?",
    "¿Qué comentarios tienen los usuarios?",
    "¿Alguien probó este modelo?",
]

#TABULAR - Preguntas sobre precios, stock, especificaciones numéricas
preguntas_tabular = [
    "¿Cuáles son las licuadoras de menos de $200?",
    "¿Qué productos cuestan entre $100 y $500?",
    "¿Cuánto stock hay de este producto?",
    "¿Qué productos están en la categoría Cocina?",
    "Quiero una licuadora con buenas reseñas y precio bajo",
    "¿Cuál es el precio de la licuadora TechHome?",
    "¿Hay stock disponible en Buenos Aires?",
    "Necesito productos de color blanco",
    "¿Qué electrodomésticos tienen garantía de 36 meses?",
    "Busco productos con potencia mayor a 1000W",
    "¿Cuál es la licuadora más barata?",
    "¿Qué productos están en oferta?",
    "¿Cuántos productos hay en stock?",
    "Quiero ver todos los productos de la marca ChefMaster",
    "¿Qué heladeras tienen capacidad mayor a 300L?",
    "Necesito un producto con peso menor a 5kg",
    "¿Cuáles son los productos más vendidos?",
    "¿Qué productos tienen descuento?",
    "Busco electrodomésticos de 220V",
    "¿Cuál es el rango de precios de licuadoras?",
]

#GRAFOS - Preguntas sobre relaciones, categorías, conexiones
preguntas_grafos = [
    "¿Qué productos están relacionados con la categoría Cocina?",
    "¿Qué vendedores venden productos de la marca TechHome?",
    "¿En qué sucursales se vende este producto?",
    "¿Qué marcas tienen productos en Climatización?",
    "¿Qué subcategorías tiene la categoría Cocina?",
    "¿Qué productos vende el vendedor V0001?",
    "¿Qué relación hay entre esta marca y esa categoría?",
    "¿Qué sucursales venden productos ChefMaster?",
    "¿Qué categorías de productos hay?",
    "¿Qué marcas están disponibles?",
    "¿Qué vendedores trabajan en Córdoba?",
    "¿Qué productos están conectados con Audio y Video?",
    "¿Qué marcas tienen productos en múltiples categorías?",
    "¿Cuántas subcategorías tiene cada categoría?",
    "¿Qué vendedores venden productos de Preparación?",
    "¿En cuántas sucursales está disponible la marca KitchenPro?",
    "¿Qué productos comparten categoría con la licuadora?",
    "¿Qué conexiones hay entre vendedores y productos?",
    "¿Qué estructura tienen las categorías?",
    "¿Qué productos pertenecen a la subcategoría Refrigeración?",
]

#Creación del dataset
for pregunta in preguntas_vectorial:
    datos_entrenamiento.append({'texto': pregunta, 'intencion': 'vectorial'})

for pregunta in preguntas_tabular:
    datos_entrenamiento.append({'texto': pregunta, 'intencion': 'tabular'})

for pregunta in preguntas_grafos:
    datos_entrenamiento.append({'texto': pregunta, 'intencion': 'grafos'})

df_train = pd.DataFrame(datos_entrenamiento)

print(f"   Distribución:")
print(df_train['intencion'].value_counts())

   Distribución:
intencion
vectorial    20
tabular      20
grafos       20
Name: count, dtype: int64


### Entrenamiento de clasificador basado en embeddings

In [396]:
X_train, X_test, y_train, y_test = train_test_split(
    df_train['texto'].values,
    df_train['intencion'].values,
    test_size=0.3,
    random_state=42,
    stratify=df_train['intencion'].values
)


#Generar embeddings usando el modelo configurado
train_embeddings = [Settings.embed_model.get_text_embedding(text) for text in X_train]
test_embeddings = [Settings.embed_model.get_text_embedding(text) for text in X_test]

train_embeddings = np.array(train_embeddings)
test_embeddings = np.array(test_embeddings)

#Entrenar clasificador
clf_trained = LogisticRegression(max_iter=1000, random_state=42)
clf_trained.fit(train_embeddings, y_train)

#Predecir
y_pred_trained = clf_trained.predict(test_embeddings)

#Métricas
accuracy_trained = accuracy_score(y_test, y_pred_trained)
print(f"   Accuracy: {accuracy_trained:.3f}")

print("\n   Reporte de clasificación:")
print(classification_report(y_test, y_pred_trained, target_names=INTENCIONES))

#Matriz de confusión
print("\n   Matriz de confusión:")
cm_trained = confusion_matrix(y_test, y_pred_trained, labels=INTENCIONES)
print(cm_trained)

   Accuracy: 0.722

   Reporte de clasificación:
              precision    recall  f1-score   support

   vectorial       0.83      0.83      0.83         6
     tabular       0.60      0.50      0.55         6
      grafos       0.71      0.83      0.77         6

    accuracy                           0.72        18
   macro avg       0.72      0.72      0.72        18
weighted avg       0.72      0.72      0.72        18


   Matriz de confusión:
[[5 1 0]
 [2 3 1]
 [0 1 5]]


### Entrenamiento de clasificador basado en LLM

In [397]:
class LLMFewShotClassifier:
    """
    Clasificador que usa Few-Shot Learning con embeddings
    Simula el comportamiento de un LLM clasificando por similitud
    """
    def __init__(self, embed_model):
        self.embed_model = embed_model
        self.examples = {
            'vectorial': [
                "¿Cómo uso mi licuadora?",
                "¿Qué dice el manual sobre mantenimiento?",
                "¿Qué opinan los usuarios sobre este producto?"
            ],
            'tabular': [
                "¿Cuáles son las licuadoras de menos de $200?",
                "¿Cuánto stock hay disponible?",
                "Quiero productos de la marca ChefMaster"
            ],
            'grafos': [
                "¿Qué productos están en la categoría Cocina?",
                "¿Qué vendedores venden productos TechHome?",
                "¿En qué sucursales se vende este producto?"
            ]
        }

        self.example_embeddings = {}
        for intent, examples in self.examples.items():
            embeddings = [embed_model.get_text_embedding(ex) for ex in examples]
            self.example_embeddings[intent] = np.array(embeddings)

    def predict(self, query: str) -> str:
        """Clasifica la query por similitud con ejemplos"""
        query_embedding = self.embed_model.get_text_embedding(query)
        query_embedding = np.array(query_embedding)

        #Calcular similitud con cada intención
        similarities = {}
        for intent, examples_emb in self.example_embeddings.items():
            sims = np.dot(examples_emb, query_embedding) / (
                np.linalg.norm(examples_emb, axis=1) * np.linalg.norm(query_embedding)
            )
            similarities[intent] = np.mean(sims)

        # Retornar intención con mayor similitud
        return max(similarities, key=similarities.get)

#Crear clasificador
clf_llm = LLMFewShotClassifier(Settings.embed_model)

#test
y_pred_llm = [clf_llm.predict(text) for text in X_test]

accuracy_llm = accuracy_score(y_test, y_pred_llm)
print(f"   Accuracy: {accuracy_llm:.3f}")

print("\n   Reporte de clasificación:")
print(classification_report(y_test, y_pred_llm, target_names=INTENCIONES))

#Matriz de confusión
print("\n   Matriz de confusión:")
cm_llm = confusion_matrix(y_test, y_pred_llm, labels=INTENCIONES)
print(cm_llm)

   Accuracy: 0.722

   Reporte de clasificación:
              precision    recall  f1-score   support

   vectorial       0.67      1.00      0.80         6
     tabular       0.75      0.50      0.60         6
      grafos       0.80      0.67      0.73         6

    accuracy                           0.72        18
   macro avg       0.74      0.72      0.71        18
weighted avg       0.74      0.72      0.71        18


   Matriz de confusión:
[[4 1 1]
 [1 3 2]
 [0 0 6]]


Ambos clasificadores tuvieron Accuracy = 72% por lo que voy a hacer la comparación entre ambos modelos


### Comparación de los clasificadores

In [398]:
print("COMPARACIÓN DE CLASIFICADORES")

#Comparación de métricas
comparacion = pd.DataFrame({
    'Métrica': ['Accuracy', 'Precision (macro)', 'Recall (macro)', 'F1-Score (macro)'],
    'Clasificador Entrenado': [
        accuracy_trained,
        0.72,
        0.72,
        0.72
    ],
    'Clasificador LLM Few-Shot': [
        accuracy_llm,
        0.74,
        0.72,
        0.71
    ]
})

print("\n", comparacion.to_string(index=False))

clasificador = clf_llm
print("\n Clasificador final seleccionado: LLM Few-Shot")

COMPARACIÓN DE CLASIFICADORES

           Métrica  Clasificador Entrenado  Clasificador LLM Few-Shot
         Accuracy                0.722222                   0.722222
Precision (macro)                0.720000                   0.740000
   Recall (macro)                0.720000                   0.720000
 F1-Score (macro)                0.720000                   0.710000

 Clasificador final seleccionado: LLM Few-Shot


## **Creación Wrapper, Pipeline y RAG**

In [399]:
#Función wrapper para usar en el pipeline
def classify_intent(query: str) -> str:
    """
    Clasifica la intención de una consulta del usuario

    Args:
        query: Consulta del usuario

    Returns:
        'vectorial', 'tabular', o 'grafos'
    """
    return clasificador.predict(query)

print("\n Función classify_intent() creada")

# Pruebas
print("PRUEBAS DEL CLASIFICADOR")
pruebas = [
    "¿Cómo uso mi licuadora para hacer smoothies?",
    "¿Cuáles son las licuadoras de menos de $300?",
    "¿Qué productos están relacionados con Cocina?",
    "¿Qué opinan los usuarios sobre esta cafetera?",
    "¿Cuánto stock hay disponible?",
    "¿Qué marcas tienen productos en Climatización?"
]

for consulta in pruebas:
    intencion = classify_intent(consulta)
    print(f"\n'{consulta}'")
    print(f"  → Intención: {intencion}")

print("\nClasificador de intenciones funcionando")


 Función classify_intent() creada
PRUEBAS DEL CLASIFICADOR

'¿Cómo uso mi licuadora para hacer smoothies?'
  → Intención: vectorial

'¿Cuáles son las licuadoras de menos de $300?'
  → Intención: tabular

'¿Qué productos están relacionados con Cocina?'
  → Intención: grafos

'¿Qué opinan los usuarios sobre esta cafetera?'
  → Intención: vectorial

'¿Cuánto stock hay disponible?'
  → Intención: tabular

'¿Qué marcas tienen productos en Climatización?'
  → Intención: grafos

Clasificador de intenciones funcionando


### Pipeline de RAG

In [400]:
print("CREANDO PIPELINE DE RAG")

# JUSTIFICACIÓN DEL LLM
justificacion_llm = """
JUSTIFICACIÓN DEL LLM:

Ubicación: API en la nube (Gemini Flash)
Modelo: gemini-1.5-flash
Razón de elección:
  1. Es GRATUITO con límite generoso (15 requests/minuto)
  2. Multilingüe (soporta español nativamente)
  3. Contexto de 1M tokens (suficiente para los chunks trabajados)
  4. Baja latencia (~1-2 segundos de respuesta)

Alternativas consideradas:
  - GPT-4: Muy costoso para este TP
  - Llama local: Requiere GPU y es más lento
  - Claude: Requiere API key de pago
"""

print(justificacion_llm)

#Mock LLM para testing
class MockLLM:
    """LLM simulado para testing sin API key"""
    def generate(self, prompt: str) -> str:
        if "Pregunta:" in prompt:
            pregunta = prompt.split("Pregunta:")[-1].strip()
        else:
            pregunta = prompt[:100]

        if "Contexto:" in prompt and prompt.split("Contexto:")[1].strip():
            return f"Basándome en la información disponible, puedo responder a tu consulta sobre: {pregunta[:50]}... La documentación indica detalles relevantes que responden tu pregunta."
        else:
            return "No encontré información específica en la base de datos. ¿Podrías reformular tu pregunta o proporcionar más detalles?"

llm = MockLLM()
print("\n LLM configurado")

CREANDO PIPELINE DE RAG

JUSTIFICACIÓN DEL LLM:

Ubicación: API en la nube (Gemini Flash)
Modelo: gemini-1.5-flash
Razón de elección:
  1. Es GRATUITO con límite generoso (15 requests/minuto)
  2. Multilingüe (soporta español nativamente)
  3. Contexto de 1M tokens (suficiente para los chunks trabajados)
  4. Baja latencia (~1-2 segundos de respuesta)

Alternativas consideradas:
  - GPT-4: Muy costoso para este TP
  - Llama local: Requiere GPU y es más lento
  - Claude: Requiere API key de pago


 LLM configurado


### Creación de sistema RAG

In [401]:
class RAGSystem:
    def __init__(self):
        self.conversation_history = []

    def retrieve(self, query: str, intent: str):
        """Recupera información según la intención"""

        if intent == 'vectorial':
            #Búsqueda híbrida
            results = hybrid_search(query, top_k=3, apply_rerank=True)
            context = "\n\n".join([
                f"[Documento {i+1}]\n{node.node.text[:500]}..."
                for i, node in enumerate(results)
            ])
            return context, "búsqueda semántica"

        elif intent == 'tabular':
            context = f"Información de productos disponible en la base de datos tabular con {len(df_productos)} productos."
            return context, "base de datos tabular"

        elif intent == 'grafos':
            context = "Información de relaciones disponible en el grafo de conocimiento."
            return context, "grafo de conocimiento"

        return "", "ninguna fuente"

    def generate_response(self, query: str) -> dict:
        """
        Genera respuesta completa para una consulta

        Returns:
            dict con 'respuesta', 'fuente', 'intencion'
        """
        #Clasificar intención
        intent = classify_intent(query)

        #Recuperar contexto
        context, source = self.retrieve(query, intent)

        #Construir prompt
        prompt = f"""Contexto: {context}

Pregunta: {query}

Instrucciones:
- Responde en español
- Usa solo la información del contexto
- Si no hay información suficiente, indica que no puedes responder
- Sé claro y conciso

Respuesta:"""

        #Generar respuesta
        response = llm.generate(prompt)

        #Guardar en memoria
        self.conversation_history.append({
            'query': query,
            'intent': intent,
            'response': response,
            'source': source
        })

        return {
            'respuesta': response,
            'fuente': source,
            'intencion': intent
        }

    def get_history(self):
        """Retorna el historial conversacional"""
        return self.conversation_history

rag_system = RAGSystem()
print("Sistema RAG creado con éxito")

Sistema RAG creado con éxito


### Ejemplos de uso del Sistema RAG

In [402]:
preguntas_prueba = [
    "¿Cómo uso mi licuadora para hacer smoothies?",
    "¿Cuáles son las licuadoras de menos de $500?",
    "¿Qué productos están relacionados con la categoría Cocina?",
]

for i, pregunta in enumerate(preguntas_prueba, 1):
    print(f"\n PRUEBA {i}:")
    print(f"Pregunta: {pregunta}")

    resultado = rag_system.generate_response(pregunta)

    print(f"\nIntención detectada: {resultado['intencion']}")
    print(f"Fuente consultada: {resultado['fuente']}")
    print(f"\nRespuesta:")
    print(resultado['respuesta'])



 PRUEBA 1:
Pregunta: ¿Cómo uso mi licuadora para hacer smoothies?

Intención detectada: vectorial
Fuente consultada: búsqueda semántica

Respuesta:
Basándome en la información disponible, puedo responder a tu consulta sobre: ¿Cómo uso mi licuadora para hacer smoothies?

Inst... La documentación indica detalles relevantes que responden tu pregunta.

 PRUEBA 2:
Pregunta: ¿Cuáles son las licuadoras de menos de $500?

Intención detectada: tabular
Fuente consultada: base de datos tabular

Respuesta:
Basándome en la información disponible, puedo responder a tu consulta sobre: ¿Cuáles son las licuadoras de menos de $500?

Inst... La documentación indica detalles relevantes que responden tu pregunta.

 PRUEBA 3:
Pregunta: ¿Qué productos están relacionados con la categoría Cocina?

Intención detectada: grafos
Fuente consultada: grafo de conocimiento

Respuesta:
Basándome en la información disponible, puedo responder a tu consulta sobre: ¿Qué productos están relacionados con la categoría... La 

In [403]:
#Buscar en la base tabular
def table_search(filtros: dict, tabla: str = 'productos') -> pd.DataFrame:
    """
    Busca en las tablas aplicando filtros dinámicos

    Args:
        filtros: Diccionario con filtros a aplicar
        tabla: 'productos' o 'inventario'

    Returns:
        DataFrame filtrado
    """
    if tabla == 'productos':
        df = df_productos.copy()
    elif tabla == 'inventario':
        df = df_inventario.copy()
    else:
        raise ValueError(f"Tabla '{tabla}' no válida")

    for columna, valor in filtros.items():
        if columna in df.columns:
            if isinstance(valor, dict):
                if 'min' in valor:
                    df = df[df[columna] >= valor['min']]
                if 'max' in valor:
                    df = df[df[columna] <= valor['max']]
            elif isinstance(valor, list):
                df = df[df[columna].isin(valor)]
            else:
                df = df[df[columna] == valor]

    return df

print("Función table_search() creada con éxito")

Función table_search() creada con éxito


# **PARTE 2**

## **Creación de Herramientas**

### TOOL 1: doc_search_tool

In [404]:
#Tool 1: Búsqueda en documentos
def doc_search_tool_func(query: str) -> str:
    """
    Busca información en manuales de productos, FAQs y reseñas de usuarios.
    Usa búsqueda semántica híbrida (vectorial + BM25 + reranking).

    Args:
        query: Pregunta sobre uso, funcionamiento o características de productos

    Returns:
        Información encontrada en los documentos
    """
    try:
        results = hybrid_search(query, top_k=3, apply_rerank=True)

        if not results:
            return "No se encontró información relevante en los documentos."

        response_parts = []
        for i, node in enumerate(results, 1):
            text_preview = node.node.text[:300].replace('\n', ' ')
            response_parts.append(f"{i}. {text_preview}...")

        return "\n\n".join(response_parts)
    except Exception as e:
        return f"Error al buscar en documentos: {str(e)}"

doc_search_tool = FunctionTool.from_defaults(fn=doc_search_tool_func)
print("doc_search_tool creada con éxito")

doc_search_tool creada con éxito


### TOOL 2: table_search_tool

In [405]:
#Tool 2: búsqueda en tablas
def table_search_tool_func(query: str) -> str:
    """Búsqueda tabular"""
    try:
        import re
        query_lower = query.lower()
        filtros = {}

        #Detectar precio
        precio_economico = any(word in query_lower for word in ['económica', 'economica', 'barata', 'barato', 'bajo precio', 'económico'])
        precio_caro = any(word in query_lower for word in ['cara', 'caro', 'premium', 'alta gama'])

        precio_match = re.search(r'menos de \$?(\d+)', query_lower)
        precio_match2 = re.search(r'más de \$?(\d+)', query_lower)

        if precio_match:
            filtros['precio_usd'] = {'max': float(precio_match.group(1))}
        elif precio_match2:
            filtros['precio_usd'] = {'min': float(precio_match2.group(1))}
        elif precio_economico:
            filtros['precio_usd'] = {'max': 300}
        elif precio_caro:
            filtros['precio_usd'] = {'min': 1000}

        #Detectar stock
        pide_buen_stock = any(word in query_lower for word in ['buen stock', 'mucho stock', 'stock alto', 'bien de stock'])

        #Detectar producto
        productos_nombres = {
            'licuadora': 'licuador',
            'cafetera': 'cafetera',
            'heladera': 'heladera',
        }

        nombre_encontrado = None
        for key, pattern in productos_nombres.items():
            if key in query_lower:
                nombre_encontrado = pattern
                break

        if nombre_encontrado:
            df_filtrado = df_productos[df_productos['nombre'].str.contains(nombre_encontrado, case=False, na=False)].copy()

            if 'precio_usd' in filtros:
                if 'max' in filtros['precio_usd']:
                    df_filtrado = df_filtrado[df_filtrado['precio_usd'] <= filtros['precio_usd']['max']]

            if pide_buen_stock:
                df_con_stock = df_filtrado[df_filtrado['stock'] >= 50].copy()

                if len(df_con_stock) > 0:
                    df_filtrado = df_con_stock
                else:
                    df_filtrado = df_filtrado.sort_values('stock', ascending=False)
                    nota = "\nNOTA: No hay productos con stock >50, mostrando los de mayor stock disponible."
            else:
                nota = ""

            if len(df_filtrado) > 0:
                df_filtrado = df_filtrado.sort_values('precio_usd')

                productos_info = []
                for _, prod in df_filtrado.head(5).iterrows():
                    productos_info.append(
                        f"- {prod['nombre']} ({prod['marca']}): ${prod['precio_usd']:.2f}, "
                        f"Stock: {int(prod['stock'])}, Categoría: {prod['categoria']}"
                    )
                return f"Encontrados {len(df_filtrado)} productos:\n" + "\n".join(productos_info) + nota
            else:
                return f"No se encontraron {nombre_encontrado}s con los criterios especificados."

        return f"Especifica el tipo de producto que buscas."

    except Exception as e:
        return f"Error: {str(e)}"

table_search_tool = FunctionTool.from_defaults(fn=table_search_tool_func)
print("table_search_tool creada con éxito")

table_search_tool creada con éxito


### TOOL 3: graph_search_tool


In [406]:
#Tool 3: búsqueda en grafos
def graph_search_tool_func(query: str) -> str:
    """Búsqueda en grafo"""
    try:
        query_lower = query.lower()

        # CRÍTICO: Buscar "marca" PRIMERO antes de verificar categoría
        if 'marca' in query_lower:
            if 'climatización' in query_lower or 'clima' in query_lower:
                cypher = """
                MATCH (p:Producto)-[:PERTENECE_A]->(c:Categoria {name: 'Climatización'}),
                      (p)-[:ES_DE_MARCA]->(m:Marca)
                RETURN DISTINCT m.name as marca
                """
                resultados = graph_search(cypher)
                if resultados:
                    items = [r['marca'] for r in resultados]
                    return f"Marcas con productos de Climatización ({len(items)}):\n" + "\n".join([f"- {item}" for item in items])

            elif 'cocina' in query_lower:
                cypher = """
                MATCH (p:Producto)-[:PERTENECE_A]->(c:Categoria {name: 'Cocina'}),
                      (p)-[:ES_DE_MARCA]->(m:Marca)
                RETURN DISTINCT m.name as marca
                """
                resultados = graph_search(cypher)
                if resultados:
                    items = [r['marca'] for r in resultados]
                    return f"Marcas con productos de Cocina ({len(items)}):\n" + "\n".join([f"- {item}" for item in items])
            else:
                cypher = """
                MATCH (m:Marca)
                RETURN DISTINCT m.name as marca
                LIMIT 10
                """
                resultados = graph_search(cypher)
                if resultados:
                    items = [r['marca'] for r in resultados]
                    return f"Marcas disponibles ({len(items)}):\n" + "\n".join([f"- {item}" for item in items])

        #verificar si pregunta por productos de una categoría
        elif 'cocina' in query_lower and ('producto' in query_lower or 'qué' in query_lower):
            cypher = """
            MATCH (p:Producto)-[:PERTENECE_A]->(c:Categoria {name: 'Cocina'})
            RETURN p.name as producto
            LIMIT 10
            """
            resultados = graph_search(cypher)
            if resultados:
                items = [r['producto'] for r in resultados]
                return f"Productos en categoría Cocina ({len(items)}):\n" + "\n".join([f"- {item}" for item in items])

        elif 'climatización' in query_lower and ('producto' in query_lower or 'qué' in query_lower):
            cypher = """
            MATCH (p:Producto)-[:PERTENECE_A]->(c:Categoria {name: 'Climatización'})
            RETURN p.name as producto
            LIMIT 10
            """
            resultados = graph_search(cypher)
            if resultados:
                items = [r['producto'] for r in resultados]
                return f"Productos en categoría Climatización ({len(items)}):\n" + "\n".join([f"- {item}" for item in items])

        elif 'sucursal' in query_lower:
            cypher = """
            MATCH (s:Sucursal)
            RETURN DISTINCT s.name as sucursal
            LIMIT 10
            """
            resultados = graph_search(cypher)
            if resultados:
                items = [r['sucursal'] for r in resultados]
                return f"Sucursales disponibles ({len(items)}):\n" + "\n".join([f"- {item}" for item in items])

        #Categorías
        else:
            cypher = """
            MATCH (c:Categoria)
            RETURN DISTINCT c.name as categoria
            """
            resultados = graph_search(cypher)
            if resultados:
                items = [r['categoria'] for r in resultados]
                return f"Categorías disponibles ({len(items)}):\n" + "\n".join([f"- {item}" for item in items])

        return "No se encontraron resultados."

    except Exception as e:
        return f"Error: {str(e)}"

graph_search_tool = FunctionTool.from_defaults(fn=graph_search_tool_func)
print("graph_search_tool creada con éxito")

graph_search_tool creada con éxito


### TOOL 4: analytics_tool_func (SQL + Gráficos)

In [407]:
engine = create_engine('sqlite:///ventas_analytics.db')

#Guardar DataFrames en SQL
df_productos.to_sql('productos', engine, if_exists='replace', index=False)
df_ventas.to_sql('ventas', engine, if_exists='replace', index=False)
df_inventario.to_sql('inventario', engine, if_exists='replace', index=False)

print("Base de datos SQL creada con 3 tablas")

#Configurar Langchain SQL Database
db = SQLDatabase(engine=engine)
print(f"   Tablas disponibles: {db.get_usable_table_names()}")

Base de datos SQL creada con 3 tablas
   Tablas disponibles: ['inventario', 'productos', 'ventas']


In [408]:
def table_search_tool_func(query: str) -> str:
    """Búsqueda tabular"""
    try:
        import re
        query_lower = query.lower()
        filtros = {}
        nota = ""

        precio_economico = any(word in query_lower for word in ['económica', 'economica', 'barata', 'barato', 'bajo precio', 'económico'])
        precio_caro = any(word in query_lower for word in ['cara', 'caro', 'premium', 'alta gama'])

        precio_match = re.search(r'menos de \$?(\d+)', query_lower)
        precio_match2 = re.search(r'más de \$?(\d+)', query_lower)

        if precio_match:
            filtros['precio_usd'] = {'max': float(precio_match.group(1))}
        elif precio_match2:
            filtros['precio_usd'] = {'min': float(precio_match2.group(1))}
        elif precio_economico:
            filtros['precio_usd'] = {'max': 300}
        elif precio_caro:
            filtros['precio_usd'] = {'min': 1000}

        pide_buen_stock = any(word in query_lower for word in ['buen stock', 'mucho stock', 'stock alto', 'bien de stock', 'con buen stock'])

        productos_nombres = {
            'licuadora': 'licuador',
            'licuadoras': 'licuador',
            'cafetera': 'cafetera',
            'cafeteras': 'cafetera',
            'heladera': 'heladera',
            'batidora': 'batidor',
            'tostadora': 'tostador',
        }

        nombre_encontrado = None
        for key, pattern in productos_nombres.items():
            if key in query_lower:
                nombre_encontrado = pattern
                break

        if nombre_encontrado:
            df_filtrado = df_productos[df_productos['nombre'].str.contains(nombre_encontrado, case=False, na=False)].copy()

            if 'precio_usd' in filtros:
                if 'max' in filtros['precio_usd']:
                    df_filtrado = df_filtrado[df_filtrado['precio_usd'] <= filtros['precio_usd']['max']]
                if 'min' in filtros['precio_usd']:
                    df_filtrado = df_filtrado[df_filtrado['precio_usd'] >= filtros['precio_usd']['min']]

            if pide_buen_stock:
                df_con_stock = df_filtrado[df_filtrado['stock'] >= 50].copy()

                if len(df_con_stock) > 0:
                    df_filtrado = df_con_stock
                    nota = ""
                else:
                    df_filtrado = df_filtrado.sort_values('stock', ascending=False)
                    nota = "\n\nNOTA: No hay productos con stock >50, mostrando los de mayor stock disponible."

            if len(df_filtrado) > 0:
                df_filtrado = df_filtrado.sort_values('precio_usd')

                productos_info = []
                for _, prod in df_filtrado.head(5).iterrows():
                    productos_info.append(
                        f"- {prod['nombre']} ({prod['marca']}): ${prod['precio_usd']:.2f}, "
                        f"Stock: {int(prod['stock'])}, Categoría: {prod['categoria']}"
                    )
                return f"Encontrados {len(df_filtrado)} productos:\n" + "\n".join(productos_info) + nota
            else:
                return f"No se encontraron {nombre_encontrado}s con los criterios especificados."

        return f"Especifica el tipo de producto que buscas."

    except Exception as e:
        return f"Error: {str(e)}"

#Actualizar la herramienta
table_search_tool = FunctionTool.from_defaults(fn=table_search_tool_func)

print("table_search_tool creada con éxito")

table_search_tool creada con éxito


In [409]:
#Generar los 4 gráficos de ejemplo
queries_test = [
    "Dame un gráfico sobre la distribución de los métodos de pago",
    "Muéstrame los productos más vendidos",
    "Análisis de ventas por provincia",
    "Ventas por sucursal"
]

print("GENERANDO LOS GRÁFICOS")


for i, query in enumerate(queries_test, 1):
    print(f"\n{i}. Ejecutando: {query}")
    resultado = analytics_tool.fn(query)
    print(f"    Completado")

print("\n Gráficos generados con éxito")
print("\nArchivos en /tmp/:")
import os
archivos = [f for f in os.listdir('/tmp/') if f.startswith('analytics_')]
for archivo in archivos:
    print(f"  - {archivo}")

GENERANDO LOS GRÁFICOS

1. Ejecutando: Dame un gráfico sobre la distribución de los métodos de pago
    Completado

2. Ejecutando: Muéstrame los productos más vendidos
    Completado

3. Ejecutando: Análisis de ventas por provincia
    Completado

4. Ejecutando: Ventas por sucursal
    Completado

 Gráficos generados con éxito

Archivos en /tmp/:
  - analytics_ventas.png
  - analytics_top_productos.png
  - analytics_provincias.png
  - analytics_sucursales.png


Adjunto las imágenes que dió como salida:

analytics_ventas.png:
https://drive.google.com/file/d/1LWYcNSrbRgZGJ4hb4d-PHv2Ut42WrZ2B/view?usp=drive_link


analytics_top_productos.png:
https://drive.google.com/file/d/1Zh5pOvkV3cIYXq_NZLP8jkFC-L69pMhk/view?usp=drive_link

analytics_provincias.png:
https://drive.google.com/file/d/1U2nXTjcgd3kN4QnjZoD-jiYK1w3RyCzn/view?usp=drive_link

analytics_sucursales.png:
https://drive.google.com/file/d/1Rx1VF-O94EcqMSyCp08bx9kK_3ROZ7vI/view?usp=drive_link


## **Creación del agente React, definición de System Prompt y memoria para contexto**

In [410]:
#Defino el system Prompt
SYSTEM_PROMPT = """
Eres un asistente experto en electrodomésticos que ayuda a los usuarios con información sobre productos.

**HERRAMIENTAS DISPONIBLES:**
- doc_search_tool: Busca en manuales, FAQs y documentación de productos
- table_search_tool: Busca productos por precio, stock, categoría, marca
- graph_search_tool: Busca relaciones entre productos, categorías y sucursales
- analytics_tool: Genera análisis y estadísticas de ventas

**REGLAS OBLIGATORIAS:**
1. SIEMPRE sigue el proceso: Thought -> Action -> Observation -> Final Answer
2. USA las herramientas apropiadas según la pregunta
3. Si la pregunta tiene múltiples partes, usa las herramientas una por una
4. NO inventes información, siempre usa las herramientas
5. Responde en español de forma clara y útil

**EJEMPLOS:**

Pregunta: ¿Cómo uso mi licuadora?
Thought: Esta pregunta es sobre uso de producto, debo buscar en documentación.
Action: doc_search_tool
Action Input: {"query": "cómo usar licuadora"}
Observation: [Información del manual]
Thought: Ya tengo la información necesaria.
Final Answer: Según el manual, para usar la licuadora...

Pregunta: ¿Cuáles son las licuadoras de menos de $300?
Thought: Esta pregunta requiere filtrar productos por precio.
Action: table_search_tool
Action Input: {"query": "licuadoras menos de 300"}
Observation: [Lista de productos]
Final Answer: Estas son las licuadoras disponibles por menos de $300...
"""

print("\n System Prompt definido")


 System Prompt definido


In [411]:
class SimpleReActAgent:
    """Agente ReAct con manejo de respuestas vacías"""

    def __init__(self, tools_list):
        self.tools = {
            'doc_search': tools_list[0],
            'table_search': tools_list[1],
            'graph_search': tools_list[2],
            'analytics': tools_list[3]
        }

    def _is_empty_response(self, observation: str) -> bool:
        """Detecta si la respuesta está vacía o no tiene información útil"""
        if not observation or len(observation.strip()) < 10:
            return True

        #Frases que indican respuesta vacía
        empty_indicators = [
            'no se encontr',
            'no hay',
            'no existe',
            'sin resultados',
            'ningún',
            'ningun',
            'error'
        ]

        observation_lower = observation.lower()
        return any(indicator in observation_lower for indicator in empty_indicators)

    def chat(self, query: str) -> str:
        """Ejecuta el ciclo ReAct: Thought -> Action -> Observation -> Answer"""

        query_lower = query.lower()

        palabras_analytics = ['análisis', 'analisis', 'top', 'más vendido', 'mas vendido',
                             'estadística', 'estadistica', 'gráfico', 'grafico', 'comparar']

        if any(palabra in query_lower for palabra in palabras_analytics):
            tool_name = 'analytics'
            tool = self.tools['analytics']
            intent = 'analytics'
        else:
            #Clasificar intención
            intent = classify_intent(query)

            if intent == 'vectorial':
                tool_name = 'doc_search'
                tool = self.tools['doc_search']
            elif intent == 'tabular':
                tool_name = 'table_search'
                tool = self.tools['table_search']
            elif intent == 'grafos':
                tool_name = 'graph_search'
                tool = self.tools['graph_search']
            else:
                tool_name = 'doc_search'
                tool = self.tools['doc_search']

        observation = tool.fn(query)

        #Verifica si la respuesta está vacía
        if self._is_empty_response(observation):
            final_answer = f"""No pude encontrar información relevante para tu consulta.

Sugerencias para reformular tu pregunta:
- Intenta ser más específico sobre lo que buscas
- Usa términos diferentes o sinónimos
- Si buscas un producto, menciona características concretas (precio, categoría, marca)
- Si buscas información de uso, especifica el producto

Ejemplos de consultas efectivas:
- "¿Cuáles son las licuadoras de menos de $300?"
- "¿Cómo usar mi licuadora para hacer smoothies?"
- "¿Qué marcas tienen productos de climatización?"

Observación técnica: {observation}"""
        else:
            final_answer = observation[:500] + ("..." if len(observation) > 500 else "")

        #Construir respuesta ReAct
        response = f"""
=== Proceso ReAct ===

Thought: Analizo la consulta "{query}".
Detecto que requiere información de tipo '{intent}'.

Action: Voy a usar la herramienta '{tool_name}' para obtener la información necesaria.

Observation:
{observation}

Final Answer:
{final_answer}

(Basado en la información encontrada usando {tool_name})
"""
        return response

#Crear agente inicial
tools = [doc_search_tool, table_search_tool, graph_search_tool, analytics_tool]
agent = SimpleReActAgent(tools)

print("Agente ReAct creado")

Agente ReAct creado


### Memoria para contexto

In [412]:
class ConversationalAgent:
    """Agente ReAct con memoria conversacional"""

    def __init__(self, base_agent):
        self.agent = base_agent
        self.history = []
        self.max_history = 5
        self.last_topic = None

    def _extract_topic(self, query: str) -> str:
        """Extrae el tema principal (sustantivos clave)"""
        query_clean = query.lower()
        query_clean = query_clean.replace('¿', '').replace('?', '')

        topics = ['licuadora', 'licuadoras', 'cafetera', 'cafeteras', 'heladera',
                 'heladeras', 'producto', 'productos', 'aire acondicionado',
                 'lavarropas', 'microondas', 'batidora', 'tostadora']

        for topic in topics:
            if topic in query_clean:
                return topic

        return None

    def _build_natural_query(self, topic: str, continuation: str) -> str:
        """
        Construye una query natural combinando tema + continuación

        Ejemplos:
        - topic="licuadora" + continuation="cuáles tienen buen stock?"
          → "licuadoras con buen stock"
        - topic="cafetera" + continuation="hay económicas?"
          → "cafeteras económicas"
        """
        continuation_clean = continuation.lower().strip()

        topic_plural = topic
        if not topic.endswith('s'):
            topic_plural = topic + 's'

        if 'buen stock' in continuation_clean or 'mucho stock' in continuation_clean:
            return f"{topic_plural} con buen stock"

        elif 'económica' in continuation_clean or 'barata' in continuation_clean:
            return f"{topic_plural} económicas"

        elif 'cara' in continuation_clean or 'premium' in continuation_clean:
            return f"{topic_plural} premium"

        elif 'mejor' in continuation_clean or 'mejores' in continuation_clean:
            return f"mejores {topic_plural}"

        else:
            continuation_clean = continuation_clean.replace('cuáles', '').replace('cuales', '')
            continuation_clean = continuation_clean.replace('qué', '').replace('que', '')
            continuation_clean = continuation_clean.strip()

            return f"{topic_plural} {continuation_clean}"

    def _enrich_query_with_context(self, query: str) -> str:
        """Enriquece la query con contexto del historial"""
        if not self.history or not self.last_topic:
            return query

        query_clean = query.lower().strip()

        continuity_indicators = ['¿y ', '¿también', 'y ', 'también', 'además', 'otro', 'otra', 'más', '¿cuáles']

        has_continuity = any(query_clean.startswith(ind.lower()) for ind in continuity_indicators)

        if has_continuity:
            query_without_prefix = query
            for ind in continuity_indicators:
                if query_clean.startswith(ind.lower()):
                    query_without_prefix = query[len(ind):].strip()
                    break

            enriched = self._build_natural_query(self.last_topic, query_without_prefix)

            print(f"\n[Memoria activa] Detectada continuidad")
            print(f"  Tema anterior: {self.last_topic}")
            print(f"  Query original: '{query}'")
            print(f"  Query enriquecida: '{enriched}'")

            return enriched

        return query

    def chat(self, query: str, use_history: bool = True) -> str:
        """Ejecuta consulta usando contexto del historial"""

        current_topic = self._extract_topic(query)

        if use_history:
            enriched_query = self._enrich_query_with_context(query)
        else:
            enriched_query = query

        response = self.agent.chat(enriched_query)

        self.history.append((query, response))

        if current_topic:
            self.last_topic = current_topic

        if len(self.history) > self.max_history:
            self.history = self.history[-self.max_history:]

        return response

    def get_history(self) -> list:
        return self.history.copy()

    def clear_history(self):
        self.history = []
        self.last_topic = None
        return "Historial limpiado"

    def show_history(self):
        if not self.history:
            return "No hay historial"

        result = "HISTORIAL:\n" + "="*70 + "\n\n"
        for i, (q, r) in enumerate(self.history, 1):
            result += f"{i}. Usuario: {q}\n   Asistente: {r[:150]}...\n\n"
        return result

conversational_agent = ConversationalAgent(agent)


### Generación con LLM

In [413]:
class LLMGenerator:
    """
    Sistema de generación de respuestas con LLM.

    NOTA IMPORTANTE:
    Por limitaciones de API key, esta versión usa un generador simplificado.
    En producción, este componente usaría Google Gemini Flash 1.5.

    JUSTIFICACIÓN DE ELECCIÓN DE LLM (para producción):
    - Modelo: Google Gemini Flash 1.5
    - Alojamiento: Nube (Google Cloud)
    - Razones:
      1. Gratuito hasta 1500 requests/día
      2. Multilingüe (excelente en español)
      3. Contexto de 1M tokens
      4. Latencia baja (~500ms)
      5. Soporta tool calling nativo

    Alternativas consideradas:
    - GPT-4: Más caro, mejor calidad
    - Claude: Excelente razonamiento, requiere pago
    - Llama 3 local: Sin costo API, requiere GPU
    """

    def __init__(self, model_name="gemini-1.5-flash"):
        self.model_name = model_name
        self.is_mock = True

    def generate(self, prompt: str, system_prompt: str = None) -> str:
        """
        Genera una respuesta usando el LLM

        En producción esto ejecutaría llamadas reales a la API de Gemini.
        Ejemplo de implementación real (requiere API key):

            from langchain_google_genai import ChatGoogleGenerativeAI

            llm = ChatGoogleGenerativeAI(
                model="gemini-1.5-flash",
                temperature=0.7,
                google_api_key=os.getenv("GOOGLE_API_KEY")
            )

            messages = [
                SystemMessage(content=system_prompt),
                HumanMessage(content=prompt)
            ]

            response = llm.invoke(messages)
            return response.content
        """

        return f"[RESPUESTA SIMULADA - En producción usaría {self.model_name}]\n{prompt[:200]}..."

    def get_info(self):
        """Retorna información sobre la configuración del LLM"""
llm_generator = LLMGenerator()

print()


## **Creación del clasificador y pruebas finales**

In [414]:
class LLMFewShotClassifier:
    """Clasificador que usa Few-Shot Learning con embeddings"""
    def __init__(self, embed_model):
        self.embed_model = embed_model
        self.examples = {
            'vectorial': [
                "¿Cómo uso mi licuadora?",
                "¿Qué dice el manual sobre mantenimiento?",
                "¿Qué opinan los usuarios sobre este producto?"
            ],
            'tabular': [
                "¿Cuáles son las licuadoras de menos de $200?",
                "¿Cuánto stock hay disponible?",
                "Quiero productos de la marca ChefMaster"
            ],
            'grafos': [
                "¿Qué productos están en la categoría Cocina?",
                "¿Qué vendedores venden productos TechHome?",
                "¿En qué sucursales se vende este producto?"
            ]
        }

        #Calcular embeddings de ejemplos
        self.example_embeddings = {}
        for intent, examples in self.examples.items():
            embeddings = [embed_model.get_text_embedding(ex) for ex in examples]
            self.example_embeddings[intent] = np.array(embeddings)

    def predict(self, query: str) -> str:
        """Clasifica la query por similitud con ejemplos"""
        query_embedding = self.embed_model.get_text_embedding(query)
        query_embedding = np.array(query_embedding)

        #Calcular similitud con cada intención
        similarities = {}
        for intent, examples_emb in self.example_embeddings.items():
            sims = np.dot(examples_emb, query_embedding) / (
                np.linalg.norm(examples_emb, axis=1) * np.linalg.norm(query_embedding)
            )
            similarities[intent] = np.mean(sims)

        return max(similarities, key=similarities.get)

clasificador = LLMFewShotClassifier(Settings.embed_model)

def classify_intent(query: str) -> str:
    """Clasifica la intención de una consulta del usuario"""
    return clasificador.predict(query)

print("\n Clasificador creado con éxito")


 Clasificador creado con éxito


### Consultas de prueba

In [415]:
consultas_prueba = [
    "¿Cómo uso mi licuadora para hacer smoothies?",
    "¿Cuáles son las licuadoras de menos de $500?",
    "¿Qué productos están en la categoría Cocina?",
    "Muéstrame un análisis de los productos más vendidos",
    "¿Qué marcas tienen productos de climatización?",
    "Necesito una cafetera económica con buen stock",
]

resultados_agente = []

for i, consulta in enumerate(consultas_prueba, 1):
    print("="*70)
    print(f"PRUEBA {i}/{len(consultas_prueba)}")
    print("="*70)
    print(f"Consulta: {consulta}\n")

    try:
        respuesta = agent.chat(consulta)
        print(respuesta)
        resultados_agente.append({
            'consulta': consulta,
            'respuesta': respuesta,
            'exito': True
        })

    except Exception as e:
        print(f"Error: {e}")
        resultados_agente.append({
            'consulta': consulta,
            'respuesta': str(e),
            'exito': False
        })

    print("\n")

print("RESUMEN DE PRUEBAS:")
exitosas = sum(1 for r in resultados_agente if r['exito'])
print(f"\nPruebas exitosas: {exitosas}/{len(consultas_prueba)}")

if exitosas == len(consultas_prueba):
    print("Todas las pruebas completadas exitosamente")
else:
    print(f"{len(consultas_prueba) - exitosas} pruebas fallaron")

PRUEBA 1/6
Consulta: ¿Cómo uso mi licuadora para hacer smoothies?


=== Proceso ReAct ===

Thought: Analizo la consulta "¿Cómo uso mi licuadora para hacer smoothies?".
Detecto que requiere información de tipo 'vectorial'.

Action: Voy a usar la herramienta 'doc_search' para obtener la información necesaria.

Observation:
1. Pregunta: ¿Cómo se usa correctamente este producto? Respuesta: El Licuadora de TechHome está diseñado para uso doméstico. Revise el manual del producto (código P0001) para más detalles. Ante cualquier duda, contacte a nuestro servicio de atención al cliente....

2. Pregunta: ¿Cómo se usa correctamente este producto? Respuesta: El Licuadora de TechHome está diseñado para uso doméstico. Revise el manual del producto (código P0001) para más detalles. Ante cualquier duda, contacte a nuestro servicio de atención al cliente....

3. Pregunta: ¿Cómo se usa correctamente este producto? Respuesta: El Licuadora de ChefMaster está diseñado para uso doméstico. Revise el manual d

### Pruebas del sistema de memoria

In [416]:
print("PRUEBAS CON MEMORIA CONVERSACIONAL")

#Secuencia de consultas que demuestran memoria
consultas_memoria = [
    "¿Cuáles son las licuadoras de menos de $300?",
    "¿Y cuáles tienen buen stock?",
    "Dame un análisis de productos más vendidos",
    "¿Qué marcas tienen productos de climatización?",
]

print("\nEjecutando secuencia de consultas con memoria...\n")

for i, consulta in enumerate(consultas_memoria, 1):
    print("="*70)
    print(f"CONSULTA {i}/{len(consultas_memoria)}")
    print("="*70)
    print(f"Usuario: {consulta}\n")

    respuesta = conversational_agent.chat(consulta)
    print(respuesta[:400])
    print("\n...")
    print(f"\n[Historial: {len(conversational_agent.history)} interacciones guardadas]\n")

#Mostrar historial completo
print("HISTORIAL COMPLETO DE LA CONVERSACIÓN")
print(conversational_agent.show_history())

PRUEBAS CON MEMORIA CONVERSACIONAL

Ejecutando secuencia de consultas con memoria...

CONSULTA 1/4
Usuario: ¿Cuáles son las licuadoras de menos de $300?


=== Proceso ReAct ===

Thought: Analizo la consulta "¿Cuáles son las licuadoras de menos de $300?".
Detecto que requiere información de tipo 'tabular'.

Action: Voy a usar la herramienta 'table_search' para obtener la información necesaria.

Observation:
Encontrados 2 productos:
- Compacto Licuadora (ChefMaster): $259.42, Stock: 75, Categoría: Cocina
- Licuadora (TechHome): $283.63, Stock: 108, C

...

[Historial: 1 interacciones guardadas]

CONSULTA 2/4
Usuario: ¿Y cuáles tienen buen stock?


[Memoria activa] Detectada continuidad
  Tema anterior: licuadora
  Query original: '¿Y cuáles tienen buen stock?'
  Query enriquecida: 'licuadoras con buen stock'

=== Proceso ReAct ===

Thought: Analizo la consulta "licuadoras con buen stock".
Detecto que requiere información de tipo 'tabular'.

Action: Voy a usar la herramienta 'table_search